In [7]:
import random
import pickle 
import pandas as pd
import numpy as np 
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM,Dense,Activation
from tensorflow.keras.optimizers import RMSprop

In [4]:
df = pd.read_csv("Fake_Real_Data.csv")
txt = list(df.Text.values)
txt = " ".join(txt)

In [5]:
part_txt = txt[:10000]

In [8]:
tokenizer = RegexpTokenizer('\w+')
tokens = tokenizer.tokenize(part_txt.lower())

In [10]:
uq_tokens = np.unique(tokens)
uq_tokens_idx = {x:i for i,x in enumerate(uq_tokens)}

In [13]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i+n_words])
    next_words.append(tokens[i+n_words])

In [18]:
x = np.zeros((len(input_words),n_words,len(uq_tokens)),dtype=bool)
y = np.zeros((len(input_words),len(uq_tokens)),dtype=bool)

In [21]:
x.shape, y.shape

((1686, 10, 656), (1686, 656))

In [23]:
for i, words in enumerate(input_words):
    for j,word in enumerate(words):
        x[i,j,uq_tokens_idx[word]] = 1
    y[i,uq_tokens_idx[next_words[i]]] = 1

In [24]:
model = Sequential()
model.add(LSTM(128,input_shape=(n_words,len(uq_tokens)),return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(uq_tokens)))
model.add(Activation("softmax"))

In [27]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(learning_rate=0.01),metrics=['accuracy'])
model.fit(x,y,batch_size=128,epochs=30,shuffle=True)

Epoch 1/30
14/14 [==============================] - 4s 37ms/step - loss: 3.8753 - accuracy: 0.1738
Epoch 2/30
14/14 [==============================] - 1s 49ms/step - loss: 2.8340 - accuracy: 0.3049
Epoch 3/30
14/14 [==============================] - 1s 67ms/step - loss: 2.2623 - accuracy: 0.4128
Epoch 4/30
14/14 [==============================] - 1s 64ms/step - loss: 1.8315 - accuracy: 0.5374
Epoch 5/30
14/14 [==============================] - 1s 62ms/step - loss: 1.4407 - accuracy: 0.6340
Epoch 6/30
14/14 [==============================] - 1s 58ms/step - loss: 1.1471 - accuracy: 0.7361
Epoch 7/30
14/14 [==============================] - 1s 59ms/step - loss: 0.8578 - accuracy: 0.8013
Epoch 8/30
14/14 [==============================] - 1s 63ms/step - loss: 0.7265 - accuracy: 0.8618
Epoch 9/30
14/14 [==============================] - 1s 65ms/step - loss: 0.5098 - accuracy: 0.9039
Epoch 10/30
14/14 [==============================] - 1s 61ms/step - loss: 0.3747 - accuracy: 0.9407
Epoch 11/

In [30]:
def predict_next_word(input_txt,n_best):
    input_txt = input_txt.lower()
    x = np.zeros((1,n_words,len(uq_tokens)))
    for i,w in enumerate(input_txt.split()):
        x[0,i,uq_tokens_idx[w]] = 1

    prediction = model.predict(x)[0]
    return np.argpartition(prediction,-n_best)[-n_best:]

In [31]:
possible = predict_next_word("I will have to look into this because",5)

1/1 [==============================] - 1s 755ms/step


In [35]:
print([uq_tokens[i] for i in possible])

['gingrich', 'women', 'r', 'after', 'told']


In [38]:
def generate_txt(input_txt,text_length,creativity=3):
    word_seq = input_txt.split()
    current = 0
    for _ in range(text_length):
        sub_seq = " ".join(tokenizer.tokenize(" ".join(word_seq).lower()))[current:current+n_words]
        try:
            choice = uq_tokens[random.choice(predict_next_word(sub_seq,creativity))]
        except:
            choice = random.choice(uq_tokens)
        word_seq.append(choice)
        current += 1
    return " ".join(word_seq)

In [39]:
generate_txt("I will have to look into this because",100,5)

1/1 [==============================] - 0s 26ms/step


'I will have to look into this because tomorrow as via which financial around reporters wi financed would least rolls committee may republican donald politician financed nation told of typical a system white rigged save if party from house savings legislation means hostile her hope rate would able purging cities small skeptical breaks tomorrow family mad chambers the u c around george estate contained they step majority told committee mean surrogates money making newt chairman rolls profits republican dollars divided scrapping some 2018 r plan plan common such focus borrowed yes fairer ground hostile rather predecessors a republican eluded would businesses raise all approve presidential some black senator'

In [40]:
model.save("mymodel.h5")